In [3]:
# Import libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sb
import numpy as np
import warnings
# from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
# from catboost import CatBoostClassifier

warnings.filterwarnings('ignore')

In [49]:
# Train Data 
data = pd.read_csv('data/data.csv')
data.head()

,id,feat0,feat1,feat2,feat3,feat4,treatment,target
0,0,0.400729,0.808413,0.943385,0.198770,-2.896355,1,906
1,1,1.115889,-2.107072,-0.512322,-0.206800,0.705603,1,558
2,2,-0.153704,-0.010682,-0.386020,0.274708,-0.277327,1,386
3,3,-1.598411,0.084529,-1.271734,-1.552409,0.819762,0,391
4,4,2.328404,-1.731663,0.069395,0.339403,0.629478,0,434


In [57]:
y_data = data[['target', 'treatment']]
y_data

,target,treatment
0,906,1
1,558,1
2,386,1
3,391,0
4,434,0
...,...,...
39995,379,0
39996,391,1
39997,1035,1
39998,447,1


In [50]:
# Определяем индексы столбцов, которые нужно извлечь (между 'id' и 'treatment')
start_column = 'id'
end_column = 'treatment'

# Извлекаем все столбцы между 'id' и 'treatment', включая 'id' и 'treatment'
X_data = data.loc[:, data.columns.str.contains('feat')] #data.loc[:, start_column:end_column]
X = X_data.values

# Извлекаем значения целевой переменной (target) в виде numpy-вектора y
y_data = data['target']
y = y_data.values

# Извлекаем флаг воздействия (treatment) в виде numpy-вектора treatment
treatment_data = data['treatment']
treatment = treatment_data.values

X_data

,feat0,feat1,feat2,feat3,feat4
0,0.400729,0.808413,0.943385,0.198770,-2.896355
1,1.115889,-2.107072,-0.512322,-0.206800,0.705603
2,-0.153704,-0.010682,-0.386020,0.274708,-0.277327
3,-1.598411,0.084529,-1.271734,-1.552409,0.819762
4,2.328404,-1.731663,0.069395,0.339403,0.629478
...,...,...,...,...,...
39995,0.357528,0.161275,0.280817,0.288936,-0.954824
39996,-0.538250,0.075060,-0.101495,-1.148672,-1.950716
39997,0.787431,0.145838,1.878684,-0.448268,-0.937813
39998,0.158120,0.591991,-2.570283,-0.726610,-2.372462


In [51]:
y_data

0         906
1         558
2         386
3         391
4         434
         ... 
39995     379
39996     391
39997    1035
39998     447
39999     607
Name: target, Length: 40000, dtype: int64

In [53]:
treatment_data

0        1
1        1
2        1
3        0
4        0
        ..
39995    0
39996    1
39997    1
39998    1
39999    0
Name: treatment, Length: 40000, dtype: int64

In [48]:
def find_threshold_options(column_values):
    unique_values = np.unique(column_values)
    if len(unique_values) > 10:
        percentiles = np.percentile(
            column_values, [3, 5, 10, 20, 30, 50, 70, 80, 90, 95, 97]
        )
    else:
        percentiles = np.percentile(unique_values, [10, 50, 90])

    threshold_options = np.unique(percentiles)
    return threshold_options


def split(X: np.ndarray, treatment: np.ndarray, y: np.ndarray, feature: int) -> tuple[float, float]:
        """Find the best split for a node (one feature) using DeltaDeltaP criterion."""
        best_threshold = None
        best_delta_delta_p = float('-inf')  # Initialize with negative infinity

        threshold_options = find_threshold_options(X[:, feature])

        for threshold in threshold_options:
            treated_mask = X[:, feature] <= threshold
            control_mask = ~treated_mask

            treated_group = y[treatment == 1][treated_mask]
            control_group = y[treatment == 0][control_mask]

            uplift_treated = np.mean(treated_group) - np.mean(control_group)

            treated_prop = np.sum(treatment == 1) / len(treatment)
            control_prop = np.sum(treatment == 0) / len(treatment)

            delta_delta_p = uplift_treated - (treated_prop - control_prop)

            if delta_delta_p > best_delta_delta_p:
                best_delta_delta_p = delta_delta_p
                best_threshold = threshold
                
        return best_threshold, best_delta_delta_p

                
split = split(X, treatment, y, feature=0)

IndexError: boolean index did not match indexed array along dimension 0; dimension is 20157 but corresponding boolean dimension is 40000

In [70]:
from collections import OrderedDict, defaultdict
from typing import Callable, Tuple, Dict, List

import numpy as np
from tqdm.auto import tqdm
import heapq

def distance(pointA: np.ndarray, documents: np.ndarray) -> np.ndarray:
    # Calculate the Euclidean distance between pointA and all documents in documents
    # Use numpy for faster calculations
    # Return an array of distances
    return np.linalg.norm(documents - pointA, axis=1, keepdims=True)

# Create a graph from points
def create_sw_graph(
        data: np.ndarray,
        num_candidates_for_choice_long: int = 10,
        num_edges_long: int = 5,
        num_candidates_for_choice_short: int = 10,
        num_edges_short: int = 5,
        use_sampling: bool = False,
        sampling_share: float = 0.05,
        dist_f: Callable = distance
    ) -> Dict[int, List[int]]:

    # Create an empty dictionary to store the graph
    sw_graph = defaultdict(list)

    # Determine the number of documents and dimensionality
    N, D = data.shape

    # Choose random points for sampling if use_sampling=True
    if use_sampling:
        num_samples = int(N * sampling_share)
        sampled_indices = np.random.choice(N, num_samples, replace=False)
    else:
        num_samples = N
        sampled_indices = np.arange(N)

    # For each point in the sample
    for i in tqdm(sampled_indices, desc="Building SW graph"):
        pointA = data[i]

        # Calculate distances from pointA to all documents
        distances = dist_f(pointA, data)

        # Get the indices of the farthest and closest points
        farthest_indices = np.argpartition(-distances, num_candidates_for_choice_long)[:num_candidates_for_choice_long]
        closest_indices = np.argpartition(distances, num_candidates_for_choice_short)[:num_candidates_for_choice_short]

        # Choose random edges for long and short connections
        long_edges = np.random.choice(farthest_indices, num_edges_long, replace=False)
        short_edges = np.random.choice(closest_indices, num_edges_short, replace=False)

        # Combine long and short edges for the current point
        all_edges = np.concatenate([long_edges, short_edges])

        # Add connections to the graph for the current point
        sw_graph[i] = list(all_edges)

    # Convert the dictionary to an OrderedDict for ease of use
    sw_graph = OrderedDict(sw_graph)

    return sw_graph

def nsw(query_point: np.ndarray,
        all_documents: np.ndarray,
        graph_edges: Dict[int, List[int]],
        search_k: int = 10,
        num_start_points: int = 5,
        dist_f: Callable = distance) -> List[int]:

    # Create a set to store search results (to prevent duplicates)
    search_results = set()

    # Choose random starting points
    start_points = np.random.choice(all_documents.shape[0], num_start_points, replace=False)

    for start_point_idx in start_points:
        start_point = all_documents[start_point_idx]

        # Initialize a min heap to store candidates and their distances
        candidates = []

        dist = np.linalg.norm(query_point - start_point)
        heapq.heappush(candidates, (dist, start_point_idx))

        # Initialize a set to keep track of visited candidates
        visited = set()

        while candidates:
            # Pop the candidate with the smallest distance
            min_dist, current_candidate_idx = heapq.heappop(candidates)

            # If the candidate has already been visited, skip it
            if current_candidate_idx in visited:
                continue

            # Mark the candidate as visited
            visited.add(current_candidate_idx)

            # Add the candidate to the search results set
            search_results.add(current_candidate_idx)

            # Calculate distances from the current candidate to its neighbors
            neighbor_indices = graph_edges[current_candidate_idx]
            neighbor_distances = dist_f(query_point, all_documents[neighbor_indices])

            # Add neighbors to candidates along with their distances
            for neighbor_idx, neighbor_dist in zip(neighbor_indices, neighbor_distances):
                heapq.heappush(candidates, (neighbor_dist, neighbor_idx))

    # Convert the set of search results back to a list
    search_results = list(search_results)

    # Sort the search results by distance (optional)
    search_results.sort(key=lambda idx: np.linalg.norm(query_point - all_documents[idx]))

    # Keep only the top search_k candidates
    if len(search_results) > search_k:
        search_results = search_results[:search_k]

    return search_results

# D = 10
# N = 10000
# np.random.seed(10)
# pointA = np.random.rand(1, D)
# documents = np.random.rand(N, D)


rand_D = 5
rand_N = 10
rand_documents = np.random.randn(rand_N, rand_D)
rand_point = np.random.randn(1, rand_D)
# print('rand_point', rand_point)


# print('dist', dist)

# sw_graph = create_sw_graph(documents)
# print(nsw(pointA, documents, sw_graph, search_k=10))


array([[-0.31594488, -1.28483815,  0.43741789, -0.98306882,  0.41093997],
       [-0.29952731, -1.03468487, -0.3663657 , -1.09295535,  1.04794894],
       [-0.45402444,  1.65728704, -1.14399673,  0.85591195,  0.96670763],
       [ 0.51658359, -1.8447982 ,  0.64641674, -0.76618062,  0.30682134],
       [-0.25735157, -1.30765275, -0.38244601, -1.21523595, -0.48515054],
       [-1.44604612,  0.80807196, -0.83409388, -1.53881463,  1.62296096],
       [-0.49016557, -0.00298644,  1.46022687,  0.67998305, -0.52985843],
       [-0.02870641, -0.07557318, -0.15574446, -1.85897035, -1.39590485],
       [ 0.9409962 ,  1.95420524, -0.41898438, -2.05920644, -2.07521716],
       [ 0.51826784, -2.2358148 ,  0.07734283, -0.41116086, -0.15741245]])

In [71]:
dist = distance(rand_point, rand_documents)
dist

array([1.49677198, 1.61356334, 3.00331844, 1.98995262, 1.33332572,
       3.05048574, 2.29553613, 2.45267443, 4.27498389, 1.81430313])